### 예제. RNN을 이용한 텍스트 생성

<p>
 Andrej Karpathy의 Referential Neural Networks의 The Unreasonable Effectiveness에서 셰익스피어가 쓴 글을 데이터 셋으로 이용하겠습니다. 문자 시퀀스 ( "Shakespear")가 주어지면 시퀀스의 다음 문자 ( "e")를 예측하기 위해 모델을 훈련시킵니다. 모델을 반복적으로 호출하여 더 긴 텍스트 시퀀스를 생성 할 수 있습니다.</p>
 
 <p>
이 튜토리얼에는 tf.keras와 eager excetuttion을 사용하여 구현하도록 하겠습니다. 일반적인 tensorflow의 연산이 나중에 실행하기 위해 계산 그래프를 만드는 반면, eager execution은 그래프를 작성하지 않고 즉시 평가하여 구체적인 값을 반환합니다. 이를 통해 TensorFlow 및 디버그 모델을 쉽게 시작할 수 있습니다. </p>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [1]:
# 탄력적으로 GPU memory 사용 방법
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

NameError: ignored

In [0]:
# 데이터셋 다운로드
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### 데이터 살펴 보기

In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
# 첫 250자 살펴보기
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
# 고유한 문자들의 수 
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


### text 처리하기
#### text를 벡터화 처리

In [0]:
# 문자->index 맵핑을 만듭니다. 
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [0]:
# 첫 13개 글자를 맵핑시키면 
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### 문자열이 주어졌을 때 다음에 어떤 문자가 올까요? 

입력으로 문자열을 받아 다음에 올 문자를 예측하는 모델을 학습시켜 보도록 하겠습니다. 

In [0]:
# 문자열의 최대 길이 지정 
seq_length = 100
examples_per_epoch = len(text)//seq_length

# training example 과 target 생성
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [0]:
# batch 를 이용하면 개별 문자를 원하는 크기의 시퀀스로 쉽게 변환 할 수 있습니다.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
# 각 시퀀스마다, map 함수를 이용하여 각 배치에 split_input_target 함수를 적용하여 텍스트를 복제 및 이동합니다.
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [0]:
"""
이들 벡터의 각 index는 하나의 시간 간격으로 처리됩니다.
시간 단계 0에서 입력에 대해 모델은 "F"에 대한 index를 받아 "i"에 대한 index를 다음 문자로 예측하려고 시도합니다.
다음 timestep에서는 동일한 작업을 수행하지만 RNN은 현재 입력 문자 이외에 이전 단계 문맥(context)를 고려합니다.
"""
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [0]:
# 배치 사이즈 설정 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# 데이터셋을 섞을 Buffer size 설정 
# (TF 데이터는 infinite 시퀀스와 함께 작동하도록 설계 되었기 때문에 
# 전체 시퀀스를 메모리에 셔플하지 않고 데이터를 섞을 버퍼를 관리합니다.

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### 모델 생성

tf.keras.Sequential를 사용하여 모델을 정의하고 세 개의 레이어를 이 예제에서 사용합니다. 

- tf.keras.layers.Embedding : 입력 레이어. embedding_dim 차원이있는 벡터에 각 문자의 숫자를 매핑하는 학습 가능한 룩업 테이블입니다. <br>
- tf.keras.layers.GRU : 사이즈가 units인 RNN의 한 종류 (여기에서 LSTM 계층을 사용할 수도 있습니다.) <br>
- tf.keras.layers.Dense : 출력 레이어. 출력은 vocab_size입니다. <br>

#### Simple RNN
<img src="https://miro.medium.com/max/306/1*28XR1ajfW1WuTOkjpOc9xA.png" width=200>

#### Gated Recurrent Unit (GRU)
<img src="https://miro.medium.com/max/436/1*GSZ0ZQZPvcWmTVatAeOiIw.png" width=250>

In [0]:
# 단어 개수
vocab_size = len(vocab)

# 문자열을 임베딩할 dimension의 수를 정합니다.
embedding_dim = 256

# RNN 유닛의 개수 
rnn_units = 1024

In [0]:
# GPU를 사용가능하다면 CuDNNGRU를 사용합니다. 
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

각 문자에 대해 모델은 임베딩을 찾아, 임베딩을 입력으로하여 GRU 하나의 타임 스텝을 실행합니다. dense 레이어를 적용하여 다음 문자의 log-likelihood을 예측한 logits를 생성합니다.

<img src="https://tensorflow.org/tutorials/sequences/images/text_generation_training.png" height=400>

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

W0718 00:02:22.574079 140052706756480 util.py:244] Unresolved object in checkpoint: (root).optimizer
W0718 00:02:22.575329 140052706756480 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer
W0718 00:02:22.579395 140052706756480 util.py:244] Unresolved object in checkpoint: (root).optimizer.global_step
W0718 00:02:22.580697 140052706756480 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta1_power
W0718 00:02:22.583320 140052706756480 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta2_power
W0718 00:02:22.585700 140052706756480 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-0.embeddings
W0718 00:02:22.589098 140052706756480 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0718 00:02:22.591246 140052706756480 util.py:244] Unresolved object in checkpoint: (root).optimizer.

### 모델 실행하기

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru_6 (CuDNNGRU)       (64, None, 1024)          3938304   
_________________________________________________________________
dense_6 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 첫번째 example에 대해 시도
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([21, 16, 47, 53,  9, 24, 58, 56,  8, 24, 37, 34, 13, 62, 20, 62, 60,
        8, 33, 11,  8, 27, 21, 16, 47, 42, 64, 49,  7,  7, 10, 59, 33, 36,
       38, 17, 40, 58,  3, 27, 12,  9,  5, 45, 50, 49,  6, 11, 29, 62, 36,
        3, 47, 32, 42, 54, 45, 60, 10,  7, 21, 52, 24, 41, 27, 42, 46, 57,
       37,  6, 64, 31, 48, 19, 15, 16, 16, 28,  7, 49, 12,  5, 50, 21, 56,
       21, 62, 17, 38, 33,  2, 64, 47, 33, 16, 22,  6, 30, 22, 39])

In [0]:
# 디코드
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'll you be gone?\n\nVIRGILIA:\n\nSICINIUS:\nAre you mankind?\n\nVOLUMNIA:\nAy, fool; is that a shame? Note bu'

Next Char Predictions: 
 "IDio3Ltr.LYVAxHxv.U;.OIDidzk--:uUXZEbt$O?3'glk,;QxX$iTdpgv:-InLcOdhsY,zSjGCDDP-k?'lIrIxEZU!ziUDJ,RJa"


### 모델 학습하기
이전 RNN state과 현재 인풋이 주어졌을 때 다음에 올 문자의 클래스를 예측합니다. 

In [0]:
# 모델이 logit를 반환하기를 원하기 때문에 from_logits를 설정합니다. 
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1746006


In [0]:
# optimizer로 Adam을 설정합니다.
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [0]:
# 학습 동안의 모델을 저장할 checkpoint를 지정합니다. 
# checkpoint가 저장될 폴더
checkpoint_dir = './training_checkpoints'
# checkpoint 파일의 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
# 학습 실행 
EPOCHS=3

history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
174/174 [==============================] - 26s 147ms/step - loss: 2.3087
Epoch 2/3
174/174 [==============================] - 24s 136ms/step - loss: 1.8610
Epoch 3/3
174/174 [==============================] - 24s 137ms/step - loss: 1.6294


In [0]:
!nvidia-smi

Thu Jul 18 00:04:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    67W / 149W |   1246MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### 텍스트 생성

In [0]:
# 마지막 checkpoint 불러오기
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_7 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_7 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


1. 시작 문자열을 선택하고, RNN state 초기화하고 생성 할 문자 수 설정합니다.
2. 시작 문자열과 RNN state을 이용해 다음에 올 문자의 분포를 예측합니다.
3. 다항 분포 (multinomial distribution)를 사용하여 예측 된 문자의 index를 계산하고, 이 예측 된 문자를 모델에 대한 다음 입력으로 사용합니다.
4. 모델에 의해 반환 된 RNN state는 모델에 들어가 이제는 단 하나의 단어가 아닌 더 많은 컨텍스트를 갖게됩니다.
다음 단어를 예측 한 후에 수정 된 RNN state가 다시 모델로 들어가 이전에 예측 된 단어에서 더 많은 컨텍스트를 얻으면서 학습하는 방식입니다.

<img src="https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png" height=300>

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      #       predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I would sot percuige!
Wo all one rithat some-possied:
I rave in amls,
If thou wilting not against us by a terron in this?

LUCENTIO:
I have pruseed you to and fallings nigrth
To a pity cheer'for feel.

CARESBUY:
And speak you? for in our lords
Syol the parters of the langs on thes and duspicion tursh princome exfosentions;
O not some how recoves my we proud,
To father me at out soothenter, 'sifum he, he shall bemover soll it
Bay, loss? our petitness of him.

Servatt:
Tell changef, by make ot nave, deceive!

DUKE VINCENTIO:
No, a thou arr: and at hingly fair ane sweet, and stird, we so must
converon, loved me well gentle against try lie.

CLARENCE:
My cursery lif-hourself and roya
Nade of yourselves be preserved
Take our worshound not to see hear ot a labben?

DUKE OF YORK:
O, yeatar failer.

Farse: But frife am so no tolu.

CARIELA:
Saye you prinent, or wordom you here stoeldst
Is wa'tly we will not fram our comfort
save us be ither, no, and away of this rishal
Ladenor, shall ep

생성 된 텍스트를 살펴보면 모델이 대문자와 단락을 언제 알 수 있는지를 알 수 있으며 셰익스피어와 유사한 글쓰기 어휘를 모방합니다. 작은 epoch에서도 일관된 문장을 형성하는 것을 조금 배웠습니다. 어떻게 결과를 향상 시킬 수 있을까요? 


- epoch을 늘린다. EPOCHS=30
- start string을 바꾸어 실험한다. 
- 다른 RNN layer를 추가한다. 
- 다른 temperature를 시도해본다. 

### Customized Training

1. RNN state 초기화. tf.keras.Model.reset_states method
2. 배치별로 데이터셋을 돌며 예측합니다.
3. tf.GradientTape 열어 예측과 손실을 계산합니다.
4. tf.GradientTape.grads 메소드를 사용하여 loss gradient를 계산합니다.
5. 옵티마이저의 tf.train.Optimizer.apply_gradients 메소드를 사용하여 더 실행합니다. 


In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()

In [0]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # initializing the hidden state at the start of every epoch
    # initially hidden is None
    hidden = model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              # feeding the hidden state back into the model
              # This is the interesting step
              predictions = model(inp)
              loss = tf.losses.sparse_softmax_cross_entropy(target, predictions)

          grads = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(grads, model.trainable_variables))

          if batch_n % 100 == 0:
              template = 'Epoch {} Batch {} Loss {:.4f}'
              print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.1757
Epoch 1 Batch 100 Loss 2.3452
Epoch 1 Loss 2.1657
Time taken for 1 epoch 26.30596113204956 sec

Epoch 2 Batch 0 Loss 2.1165
Epoch 2 Batch 100 Loss 1.9056
Epoch 2 Loss 1.8287
Time taken for 1 epoch 27.32648515701294 sec

Epoch 3 Batch 0 Loss 1.7810
Epoch 3 Batch 100 Loss 1.6569
Epoch 3 Loss 1.6253
Time taken for 1 epoch 25.935643434524536 sec

Epoch 4 Batch 0 Loss 1.5970
Epoch 4 Batch 100 Loss 1.5132
Epoch 4 Loss 1.5052
Time taken for 1 epoch 26.1180682182312 sec

Epoch 5 Batch 0 Loss 1.4850
Epoch 5 Batch 100 Loss 1.4261
Epoch 5 Loss 1.4253
Time taken for 1 epoch 26.037880182266235 sec

Epoch 6 Batch 0 Loss 1.4080
Epoch 6 Batch 100 Loss 1.3636
Epoch 6 Loss 1.3658
Time taken for 1 epoch 25.981739044189453 sec

Epoch 7 Batch 0 Loss 1.3486
Epoch 7 Batch 100 Loss 1.3130
Epoch 7 Loss 1.3154
Time taken for 1 epoch 25.879635334014893 sec

Epoch 8 Batch 0 Loss 1.2993
Epoch 8 Batch 100 Loss 1.2675
Epoch 8 Loss 1.2683
Time taken for 1 epoch 25.921844482421875 sec

Epoc